In [ ]:
import os
import shutil

In [ ]:
try:
    os.mkdir('zip')
except:
    pass
os.chdir('zip')
try:
    os.mkdir('../pdf_new')
except:
    pass
for folder in os.listdir(os.getcwd()):
    if folder.endswith('.pdf'):
        shutil.move(folder,os.getcwd()[:-3]+'pdf_new/'+folder)
    elif folder.endswith('ipynb'):
        continue
    else:            
        for file in os.listdir(os.getcwd()+'/'+folder):
            shutil.move(os.getcwd()+'/'+folder+'/'+file,os.getcwd()[:-3]+'pdf_new/'+file)
        os.rmdir(folder)
os.chdir('../') 
os.rmdir('zip')

In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from csv import writer

In [57]:
#create folder to save csv files
month=str(datetime.now().month)
year=str(datetime.now().year)
try:
    os.mkdir('csv_'+month+'_'+year)
except:
    pass
#open main page of EHPA
manufacturers_info = BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/?type=109126').content, 'html.parser')
#look for all manufacturers
for manufacturer_info in manufacturers_info.find_all('td'):                                                               
    manufacturer=manufacturer_info.text
    models_info= BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/'+manufacturer_info.a.get('href')).content, 'html.parser')
    #look for models:
    for model in models_info.find_all('td'):
        if model.text!=manufacturer:
            try:
                model_info = BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/'+model.a.get('href')).content, 'html.parser')
            except:
                continue
            for info_col in model_info.find_all(class_='info-coll'):
                if (info_col.find(class_='info-label').span.text)==('Refrigerant'):
                    ref = (info_col.find(class_='info-data').text.replace(' ','').replace('\n',''))
                if (info_col.find(class_='info-label').span.text)==('Mass of Refrigerant'):
                    mass_of_ref = (info_col.find(class_='info-data').text.replace('\n',''))
                if (info_col.find(class_='info-label').span.text)==('Certification Date'):
                    date = (info_col.find(class_='info-data').text.replace(' ','').replace('\n',''))
            #choose correct export method (csv preffered) and get basic data TODO
            for export_method in model_info.find_all('a'):
                if export_method.text.startswith('Export'):
                    soup3 = BeautifulSoup(requests.get('https://www.heatpumpkeymark.com/'+export_method.get('href')).content,'html.parser')
                    #open Download link
                    for link in soup3.find_all('a'):
                        if link.text=='Download':
                            #write to csv
                            foldername='csv_'+month+'_'+year+'/'
                            filename=manufacturer+model.text.replace('/','_')
                            csv_content = requests.get('https://www.heatpumpkeymark.com/'+link.get('href')).content
                            with open(foldername+filename+'.csv', 'wb') as file:
                                file.write(csv_content)
                            with open(foldername+filename+'.csv', 'a') as f:
                                f.write('"","Refrigerant","'+str(ref)+'","0","0","0","0"\r\n')
                                f.write('"","Mass of Refrigerant","'+str(mass_of_ref)+'","0","0","0","0"\r\n')
                                f.write('"","Date","'+str(date)+'","0","0","0","0"\r\n')

In [114]:
import pandas as pd

In [119]:
pd.read_csv('first_heatpump.csv')

,modelID,varName,value,temperature,climate,indoorUnittype,info
0,NaN,title,Acond PRO-R,0,0,0,1
1,NaN,application,2,0,0,0,1
2,NaN,indoor,1,0,0,0,1
3,NaN,outdoor,1,0,0,0,1
4,NaN,coolingMode7c12c,0,0,0,0,1
...,...,...,...,...,...,...,...
177,NaN,EN14825_027,0,5,3,0,2
178,NaN,EN14825_028,0.00,4,3,0,2
179,NaN,EN14825_028,0.00,5,3,0,2
180,NaN,EN14825_029,4246,4,3,0,2
